# Nucleophilic aromatic substitution (SNAr) reaction

<div><img src="https://pubs.rsc.org/image/article/2017/re/c6re00109b/c6re00109b-s1_hi-res.gif" width="600"/></div>

This case aims to model a SNAr reaction. There are five species involved in the process:
1. 2,4-difluoronitrobenzene (dfnb)
2. pyrrolidine (prld)
3. ortho-substituted product (ortho)
4. para-substituted product (para)
5. bis-adduct product (bis)  

2,4-difluoronitrobenzene 1 reacts with pyrrolidine 2 in ethanol to give a mixture: desired ortho-substituted product 3, para-substituted side product 4 and bis-adduct side product 5. Concenration data of the target molecule 3 is collected and used to build the process model. In this case, neither reactants nor products are measured for reaction 4, of which the kinetics parameters cannot be identified.


The process setup is shown below. Temperature, residence time and pyrrolidine concentration can be tuned to maximise the yield of ortho-substituted product 3.

<div><img src="https://pubs.rsc.org/image/article/2017/re/c6re00109b/c6re00109b-f1_hi-res.gif" width="600"/></div>

More details can be found at:
[C.A. Hone, N. Holmes, G.R. Akien, R.A. Bourne, F.L. Muller, Rapid multistep kinetic model generation from transient flow data, React. Chem. Eng. 2 (2017) 103–108. https://doi.org/10.1039/C6RE00109B](https://doi.org/10.1039/C6RE00109B)


### Parameter Setup
In this exercise, all parameters are represented with name and indicators of solid/gas, stream, reaction, and species dimensions.  
Parameter is given as `(Parameter Name, Solid/Liquid index, Stream index, Reaction index, Species index)`  
Note that `None` will be applied if the parameter is not associated with that specific dimension.  

Examples are like:

> **Operation parameters**
>
> - ("Temperature", None, None, None, None)     [No dimension related to temperature]
> - ("Residence_Time", None, None, None, None)  [No dimension related to residence time]
> - ("Concentration", None, 0, None, 1)         [Concentration of pyrrolidine]  
>  
> **Kinetics parameters**
> - ("Activation_Energy", None, 0, 0, None)                 [Activation energy is related to stream and reaction dimensions]
> - ("Referenced_Reaction_Rate_Constant", None, 0, 0, None) [Referenced reaction rate constant is related to stream and reaction dimensions]

### Unit Setup

Parameter setup can be found in the method `var2unit`

| Parameter                         | Unit   |
| --------------------------------- | ------ |
| Referenced_Reaction_Rate_Constant |        |
| Activation_Energy                 | kJ/mol |
| Temperature                       | oC     |
| Concentration                     | mol/L  |
| Residence_Time                    | min    |

In [1]:
# import required python libraries
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from bayes_opt import BayesianOptimization, acquisition
from plotly.offline import init_notebook_mode

from cases.pyrrolidine_snar import PyrrolidineSNAr

init_notebook_mode(connected=True)

# define phenomenon for the process to define the model
phenos = {
    "Mass accumulation":    "Continuous",
    "Flow pattern":         "Tubular_Flow",
    "Mass transport":       [],
}

pyrrolidine_snar = PyrrolidineSNAr(phenos, random_seed=42)

### Model Simulation
In this section, we run simulation for the SNAr reaction with published kinetic parameters.  
Concetration landscapes can be obtained by running simulation on the mesh grid of operation parameters.

In [2]:
# list of operation parameters
# 1 dfnb concentration is fixed to 0.2, other operation parameters are to be optimised to maximise the yield of 3 ortho
pyrrolidine_snar.operation_params()

{('Concentration', None, 0, None, 0): 0.2,
 ('Concentration', None, 0, None, 1): None,
 ('Temperature', None, None, None, None): None,
 ('Residence_Time', None, None, None, None): None}

In [3]:
# model simulation and plot concentration profiles
# the target product 3 ortho accumulates immediately after the reaction starts
# after most 1 dfnb has been reacted, side reaction 3 becomes faster than reaction 1, which consumes the produced 3 ortho 
operation_params = {
    ("Temperature", None, None, None, None): 100,
    ("Residence_Time", None, None, None, None): 1,
    ("Concentration", None, 0, None, 1): 0.4,
}
pyrrolidine_snar.plot_simulation_profiles(operation_params)

In [4]:
# Target product concentration profiles under varied temperatures
# At a lower temperature, the reaction rate is slow and 1 dfnb is enough for reaction 1 to compete with reaction 3
# At a higher temperature, side reaction 3 can decrease the concentration of ortho 3 after it reaches the highest concentration
operation_params = {
    ("Temperature", None, None, None, None): [30.0, 60.0, 90.0, 120.0],
    ("Residence_Time", None, None, None, None): 1,
    ("Concentration", None, 0, None, 1): 0.4,
}
pyrrolidine_snar.plot_product_profile_with_temperatures(operation_params)

In [5]:
# Target product concentration profiles under varied pyrrolidine concentrations
# Similar thing happens with the pyrrolidine concentration. 3 ortho concentration will go lower at the latter flow reactor part if 2 pyrrolidine is excessive
operation_params = {
    ("Temperature", None, None, None, None): 100,
    ("Residence_Time", None, None, None, None): 1,
    ("Concentration", None, 0, None, 1): [0.1, 0.2, 0.3, 0.4, 0.5],
}
pyrrolidine_snar.plot_product_profile_with_prld_concs(operation_params)

In [6]:
# Target product concentration landscapes
# The optimal operation points gradually slide to regions with lower residence time and lower 3 prld concentration to avoid side reaction
operation_params = {
    ("Temperature", None, None, None, None): np.linspace(30, 120, 3),
    ("Residence_Time", None, None, None, None): np.linspace(0.5, 2, 16),
    ("Concentration", None, 0, None, 1): np.linspace(0.1, 0.5, 21),
}
pyrrolidine_snar.plot_product_conc_landscapes(operation_params)

### Model calibration
In this section, we calibrate reaction kinetics parameters of the SNAr reaction.  

Arrhenius equation can be given as: $k = A \cdot exp(-\frac{E_a}{RT})$  
In the model, Arrhenius equation is reparemeterised, using a reference temperature of 90 oC; this allows to remove pre-activation constant from regression: $k = k_{ref} \cdot exp(-\frac{E_a}{RT}(\frac{1}{T} - \frac{1}{363.15}))$  
The reaction rate constant referenced to 90 oC ($k_{ref}$) and activation energy ($E_a$) are kinetics parameters to be calibrated in this case  

For Arrhenius, $ln(k)$ is linear with $\frac{1}{T}$  
For example, $ln(k)$ of $2NO_2 \rightarrow 2NO + O_2$ is measured and fitted with $\frac{1}{T}$ as the below figure
<div><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Arrhenius_plot_with_break_in_y-axis_to_show_intercept.svg/500px-Arrhenius_plot_with_break_in_y-axis_to_show_intercept.svg.png" width="600"/></div>

In [7]:
# print mapping from index to name for measure parameters
print(pyrrolidine_snar.measure_ind2name())

# print mapping from name to index for operation parameters
print(pyrrolidine_snar.operation_name2ind())

{('Concentration', None, 0, None, 2): 'outlet_ortho_conc'}
{'prld_conc': ('Concentration', None, 0, None, 1), 'temp': ('Temperature', None, None, None, None), 't_r': ('Residence_Time', None, None, None, None)}


### Latin Hypercude Sampling
Latin Hypercube Sampling is a space-filling sampling technique that improves efficiency and uniformity over simple random sampling. It is widely used in Monte Carlo simulation, because it can drastically reduce the number of runs necessary to achieve a reasonably accurate result.  

![LHS](https://www.researchgate.net/profile/Jovica-Milanovic/publication/276113319/figure/fig11/AS:981852236021760@1611103223017/Comparison-of-random-and-Latin-hypercube-sampling-examples-in-two-dimensions-every-row_W640.jpg)  

See [Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.qmc.LatinHypercube.html) for implementation details

In [8]:
# generate dataset via latin hypercube sampling and add a bit of noise (±2%)
# Latin Hypercube Sampling (LHS) is a method to produce good coverage of the parameter space with a few samples
# here, we specify the design range of each operation parameter and number of samples to the LHS method
operation_param_ranges = {
    "prld_conc":    (0.1, 0.5), # mol/L
    "temp":         (30, 120),  # oC
    "t_r":          (0.5, 2),   # min
}
dataset = pyrrolidine_snar.generate_lhs_dataset(operation_param_ranges, 7)
dataset["outlet_ortho_conc"] *= (1 + (np.random.random(size=7) - 0.5) / 25)
dataset

,prld_conc,temp,t_r,outlet_ortho_conc
0,0.447615,82.573029,1.812159,0.163292
1,0.425182,30.582993,0.676765,0.121827
2,0.100193,58.902427,1.753490,0.090419
3,0.363809,99.694746,1.533830,0.159256
4,0.200036,69.498478,1.046334,0.151458
5,0.266829,43.770331,0.925445,0.132497
6,0.285996,113.466263,1.322613,0.170097


### Prediction error minimisation
To calibrate model parameters, we do experiments (here run simulation with noise) to collect concentration information  
Then, concentration is predicted using the model under those experimental condictions  
Error is calculated and minimised by fitting kinetics or mass transport parameters  
In this case, target product ortho 3 is measured and mean squared error is used as the error:  
$Error = Mean((c_{measured,3} - c_{predicted,3})^2)$
The quasi-Newton algorithm `L-BFGS-B` is used to minimise this error  
  
To understand the realisation:
1. Go to `cases/pyrrolidine_snar.py` and locate the call of `minimize` in the `calibrate` method  
2. Check the function for calculating error `calc_mse` there  
3. Following that, `minimize` is called using the `L-BFGS-B` method  
4. Other minimisation algorithms are accessible, such as `Nelder-Mead` and `Powell`

In [9]:
# calibrate kinetics parameters by minimising the mean squared error of ortho product
cal_kinetics_param_ranges = {
    ("Referenced_Reaction_Rate_Constant", None, 0, 0, None):    (1e-4, 1e0),
    ("Referenced_Reaction_Rate_Constant", None, 0, 1, None):    (1e-4, 1e0),
    ("Referenced_Reaction_Rate_Constant", None, 0, 2, None):    (1e-4, 1e0),
    ("Referenced_Reaction_Rate_Constant", None, 0, 3, None):    (1e-4, 1e0),
    ("Activation_Energy", None, 0, 0, None):                    (20, 60),
    ("Activation_Energy", None, 0, 1, None):                    (20, 60),
    ("Activation_Energy", None, 0, 2, None):                    (20, 60),
    ("Activation_Energy", None, 0, 3, None):                    (20, 60),
}
cal_kinetics_params = pyrrolidine_snar.calibrate(cal_kinetics_param_ranges, dataset)
cal_kinetics_params

{('Referenced_Reaction_Rate_Constant', None, 0, 0, None): 0.936456,
 ('Referenced_Reaction_Rate_Constant', None, 0, 1, None): 0.084794,
 ('Referenced_Reaction_Rate_Constant', None, 0, 2, None): 0.007102,
 ('Referenced_Reaction_Rate_Constant', None, 0, 3, None): 0.598998,
 ('Activation_Energy', None, 0, 0, None): 39.992357,
 ('Activation_Energy', None, 0, 1, None): 40.002747,
 ('Activation_Energy', None, 0, 2, None): 40.002287,
 ('Activation_Energy', None, 0, 3, None): 40.000537}

In [10]:
# compare ortho product concentration landscape of the calibrated model against the ground-truth at 100 oC
# the two product concentration landscapes overlap with each other, indicating kinetics parameters are well fitted
operation_params = {
    ("Temperature", None, None, None, None): 100,
    ("Residence_Time", None, None, None, None): np.linspace(0.5, 2, 16),
    ("Concentration", None, 0, None, 1): np.linspace(0.1, 0.5, 21),
}
pyrrolidine_snar.plot_product_conc_landscape_with_ground_truth(operation_params, cal_kinetics_params)

### Surrogate model and Bayesian optimisation
Bayesian Optimisation is a global optimization strategy for expensive, black-box functions  
It uses a **surrogate model (typically Gaussian Process)** + an **acquisition function** to decide where to sample next  
Bayesian Optimisation is usually adopted when:
- Experiments are hard to evaluate
- Model is Black-box
- only a limited number of evaluations are affordable
  
What is Gaussian Process (GP):  
A Gaussian Process is a non-parametric probabilistic model that defines a distribution over functions  
It assumes that any collection of function values has a joint Gaussian distribution  
GP gives predictions + uncertainty, which the acquisition function uses to choose the next evaluation point
  
What is acquisition function:  
An acquisition function is a decision rule that tells Bayesian Optimisation where to sample next  
It balances prediction and uncertainty, some acquisition function:
| Name | Formula | Behaviour |
| ---- | ------- | --------- |
| Expected Improvement (EI) | $\mathbb{E}[\max(0, f(x) - f^+)]$ | Prefers points likely to improve the current best |
| Probability of Improvement (PI) | $P(f(x) > f^+)$ | Focuses on likely improvements (less exploratory) |
| Upper Confidence Bound (UCB) | $\mu(x) + \kappa \sigma(x)$ | Balances exploration (σ) and exploitation (μ) with parameter κ |

More details can be found in this book [Bayes Opt Book](https://bayesoptbook.com/book/bayesoptbook.pdf)  
In this SNAr case, we adopt the [bayesian-optimization](https://github.com/bayesian-optimization/BayesianOptimization) package

In [11]:
# set parameter bounds
pbounds={
    "prld_conc":    (0.1, 0.5), # mol/L
    "temp":         (30, 120),  # oC
    "t_r":          (0.5, 2),   # min
}

# define acquisition function as UCB
acq = acquisition.UpperConfidenceBound(kappa=1.5)

# define Bayesian optimization instance
optimiser = BayesianOptimization(f=None, acquisition_function=acq, pbounds=pbounds, random_state=0)

# register LHS data to Bayesian optimization
for i in range(len(dataset)):
    x = {
        k: dataset.loc[i, k]
        for k in ["prld_conc", "temp", "t_r"]
    }
    y = dataset.loc[i, "outlet_ortho_conc"]
    optimiser.register(x, y)

| 1         | 0.1632920 | 0.4476146 | 82.573028 | 1.8121587 |
| 2         | 0.1218271 | 0.4251818 | 30.582993 | 0.6767654 |
| 3         | 0.0904188 | 0.1001926 | 58.902426 | 1.7534897 |
| 4         | 0.1592560 | 0.3638090 | 99.694745 | 1.5338303 |
| 5         | 0.1514584 | 0.2000363 | 69.498477 | 1.0463344 |
| 6         | 0.1324967 | 0.2668286 | 43.770330 | 0.9254454 |
| 7         | 0.1700970 | 0.2859960 | 113.46626 | 1.3226126 |


In [12]:
# run Bayesian optimisation for 5 iterations
# operation conditions with a higher yield is found by Bayesian optimisation
np.random.seed(42)
for i in range(5):
    # sample the next suggested point and convert it to pandas tabular data
    next_suggested_point = optimiser.suggest()
    next_suggested_point = pd.DataFrame({
        k: [v] for k, v in next_suggested_point.items()
    })

    # run the experiment (in silico) and add ±2% noise
    exp_dataset = pyrrolidine_snar.run_dataset(next_suggested_point)
    exp_dataset["outlet_ortho_conc"] *= (1 + (np.random.random() - 0.5) / 25)

    # register the new experimental data to Bayesian optimization
    for i in range(len(exp_dataset)):
        x = {
            k: exp_dataset.loc[i, k]
            for k in ["prld_conc", "temp", "t_r"]
        }
        y = exp_dataset.loc[i, "outlet_ortho_conc"]
        optimiser.register(x, y)

| 8         | 0.1433252 | 0.3583576 | 119.66410 | 1.5840867 |
| 9         | 0.1817049 | 0.2264965 | 111.24377 | 1.4425205 |
| 10        | 0.1698677 | 0.5       | 107.05216 | 0.5       |
| 11        | 0.0869863 | 0.1       | 90.440364 | 0.5       |
| 12        | 0.1577233 | 0.5       | 77.530307 | 2.0       |


In [13]:
# illustratively plot heatmaps for prediction and uncertainty (prld_conc = 0.5)
temperatures = np.linspace(30, 120, 19)
residence_times = np.linspace(0.5, 2, 16)
prld_conc = 0.5
test_dataset = {"prld_conc": [], "temp": [], "t_r": []}
for temperature in temperatures:
    for residence_time in residence_times:
        test_dataset["prld_conc"].append(prld_conc)
        test_dataset["temp"].append(temperature)
        test_dataset["t_r"].append(residence_time)
test_dataset = pd.DataFrame(test_dataset)
mu, std = optimiser._gp.predict(test_dataset.values, return_std=True)
test_dataset["outlet_ortho_conc_mu"] = mu
test_dataset["outlet_ortho_conc_std"] = std

In [14]:
# plot the prediction landscape
shape = (len(temperatures), len(residence_times))
fig = go.Figure(
    data=[
        go.Surface(
            x=test_dataset["temp"].values.reshape(shape), 
            y=test_dataset["t_r"].values.reshape(shape),  
            z=test_dataset["outlet_ortho_conc_mu"].values.reshape(shape), 
            coloraxis="coloraxis",
        )
    ]
)
fig.update_layout(
    scene=dict(
        xaxis=dict(tickmode="array", tickvals=[30, 60, 90, 120], title="Temperature (oC)"),
        yaxis=dict(tickmode="array", tickvals=[0.5, 1, 1.5, 2], title="Residence time (min)"),
        zaxis=dict(tickmode="array", tickvals=[0.05, 0.1, 0.15, 0.2], title="3 Ortho conc pred (M)"),
    ),
    coloraxis=dict(colorscale="Viridis", cmin=0.06, cmax=0.18),
    width=900, height=700,
    scene_camera = dict(eye=dict(x=1.5, y=1.5, z=1.5)),
    title="Product Concentration Prediction"
)
fig.show()

In [15]:
# plot the uncertainty landscape
shape = (len(temperatures), len(residence_times))
fig = go.Figure(
    data=[
        go.Surface(
            x=test_dataset["temp"].values.reshape(shape), 
            y=test_dataset["t_r"].values.reshape(shape),  
            z=test_dataset["outlet_ortho_conc_std"].values.reshape(shape), 
            coloraxis="coloraxis",
        )
    ]
)
fig.update_layout(
    scene=dict(
        xaxis=dict(tickmode="array", tickvals=[30, 60, 90, 120], title="Temperature (oC)"),
        yaxis=dict(tickmode="array", tickvals=[0.5, 1, 1.5, 2], title="Residence time (min)"),
        zaxis=dict(tickmode="array", tickvals=[0, 0.01, 0.02, 0.03], title="3 Ortho conc uncertainty (M)"),
    ),
    coloraxis=dict(colorscale="Viridis", cmin=0, cmax=0.03),
    width=900, height=700,
    scene_camera = dict(eye=dict(x=1.5, y=1.5, z=1.5)),
    title="Product Concentration Uncertainty"
)
fig.show()